# Monitoramento de Notícias sobre Inflação de Alimentos

Este notebook monitora feeds RSS em busca de notícias relevantes sobre inflação de alimentos, utilizando palavras-chave específicas. O monitoramento é realizado em intervalos regulares.

---

## Importação de Bibliotecas

Nesta célula, importamos as bibliotecas necessárias para o monitoramento.

In [1]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime

In [2]:
import re

def search_keywords_in_rss(feed_urls, keywords):
    matches = []
    
    headers = {'User-Agent': 'Mozilla/5.0 (compatible; RSSBot/1.0)'}

    # Pré-compila expressões regulares para cada keyword
    keyword_patterns = {
        keyword: re.compile(rf'\b{re.escape(keyword.lower())}\b', re.IGNORECASE)
        for keyword in keywords
    }

    for url in feed_urls:
        print(f"Analisando feed: {url}")
        try:
            response = requests.get(url, headers=headers, timeout=10)

            if response.status_code == 200:
                try:
                    root = ET.fromstring(response.content)
                except ET.ParseError as e:
                    print(f"Erro de parse XML no feed {url}: {e}")
                    continue

                items = root.findall('.//item')
                print(f"Foram encontrados {len(items)} itens no feed.")

                for item in items:
                    title = item.find('title').text if item.find('title') is not None else ""
                    description = item.find('description').text if item.find('description') is not None else ""
                    link = item.find('link').text if item.find('link') is not None else ""
                    pub_date = item.find('pubDate').text if item.find('pubDate') is not None else ""

                    content = f"{title} {description}".lower()

                    for keyword, pattern in keyword_patterns.items():
                        if pattern.search(content):
                            matches.append({
                                'title': title,
                                'description': description,
                                'link': link,
                                'pub_date': pub_date,
                                'feed_url': url,
                                'matched_keyword': keyword
                            })
                            break  # evita duplicação

            else:
                print(f"Falha ao acessar feed: {url}, status: {response.status_code}")

        except requests.RequestException as e:
            print(f"Erro ao tentar acessar {url}: {e}")

    return matches


In [3]:
rss_feed_url = [
    # Feeds funcionando corretamente:
    'https://g1.globo.com/rss/g1/',  # Portal de notícias da Globo, ampla cobertura nacional, política, economia e cotidiano.
    'https://feeds.folha.uol.com.br/emcimadahora/rss091.xml',  # Folha de S.Paulo, jornal com ampla cobertura em política, economia e sociedade.
    'https://www.bbc.com/portuguese/index.xml',  # Versão em português da BBC, com ampla cobertura internacional e local.
    'https://exame.com/feed/',  # Revista Exame, especializada em economia, negócios e investimentos.
    'https://www.cartacapital.com.br/feed/',  # Carta Capital, revista focada em política, economia e sociedade.
    'https://www.istoedinheiro.com.br/feed/',  # Revista especializada em economia e negócios.
    'https://www.infomoney.com.br/feed/',  # Site especializado em economia, mercado financeiro e investimentos.
    'https://www.jovempan.com.br/feed',  # Portal da Rádio Jovem Pan, com cobertura em tempo real sobre política, economia e cotidiano.
    'https://economia.ig.com.br/rss.xml',  # Portal iG, notícias gerais com foco em economia e negócios.

    # # Feeds sem itens (provável sitemap ou estrutura não compatível):
    # 'https://www.cnnbrasil.com.br/sitemap-news.xml',  # CNN Brasil, notícias rápidas e cobertura internacional.
    # 'https://oglobo.globo.com/rss.xml',  # Jornal O Globo, notícias nacionais, internacionais, política e economia.

    # # Feeds com erro de parsing (XML inválido):
    # 'https://rss.uol.com.br/feed/noticias.xml',  # Portal UOL, um dos maiores portais com notícias gerais e economia.
    # 'https://www.terra.com.br/rss/Controller?channelid=3d5d59942b25e410VgnVCM10000098cceb0aRCRD&ctName=atomo-noticia',  # Portal Terra, notícias gerais, economia e cotidiano.

    # # Feeds com status HTTP 404 (não encontrados):
    # 'https://www.estadao.com.br/rss/ultimas.xml',  # O Estado de S. Paulo, tradicional jornal brasileiro focado em política e economia.
    # 'https://noticias.r7.com/feed.xml',  # Portal da Record com notícias gerais, economia e política.
    # 'https://brasil.elpais.com/rss/brasil/portada.xml',  # Versão brasileira do jornal espanhol El País, com foco em análise política.
    # 'https://veja.abril.com.br/rss.xml',  # Revista Veja, foco em política, economia e atualidades.
    # 'https://epocanegocios.globo.com/rss/ultimas/feed.xml',  # Época Negócios, revista de negócios e economia.
    # 'https://www.correiobraziliense.com.br/rss/noticia-brasil.xml',  # Correio Braziliense, jornal tradicional de Brasília com foco político e econômico.

    # # Feeds com falha de conexão:
    # 'https://www.valor.globo.com/rss',  # Valor Econômico, especializado em economia, mercados financeiros e negócios.
]
print(rss_feed_url)

['https://g1.globo.com/rss/g1/', 'https://feeds.folha.uol.com.br/emcimadahora/rss091.xml', 'https://www.bbc.com/portuguese/index.xml', 'https://exame.com/feed/', 'https://www.cartacapital.com.br/feed/', 'https://www.istoedinheiro.com.br/feed/', 'https://www.infomoney.com.br/feed/', 'https://www.jovempan.com.br/feed', 'https://economia.ig.com.br/rss.xml']


In [4]:
import pandas as pd

# Função para ler arquivo CSV e criar lista de keywords adicionais
def load_keywords_from_csv(csv_filepath):
    df = pd.read_csv(csv_filepath)

    # Removendo duplicatas e NaNs, convertendo para uma única lista
    keywords_adicionais = pd.unique(
        df[['categoria_grupo', 'categoria_subgrupo', 'categoria_item', 'categoria_subitem']].values.ravel()
    )

    # Removendo possíveis valores NaN e convertendo para lista
    keywords_adicionais = [str(keyword).strip() for keyword in keywords_adicionais if pd.notna(keyword)]

    return keywords_adicionais

# Exemplo de uso
csv_filepath = 'data/keywords/ipca_alimentacao_bebidas.csv'
keywords_adicionais = load_keywords_from_csv(csv_filepath)

# Keywords já existentes
keywords = ['inflação', 'preço dos alimentos', 'alta dos preços', 'IPCA', 'alimentação', 'bebidas']

# Combinando as listas de keywords
keywords.extend(keywords_adicionais)

print(keywords)


['inflação', 'preço dos alimentos', 'alta dos preços', 'IPCA', 'alimentação', 'bebidas', 'Alimentação e bebidas', 'Alimentação fora do domicílio', 'Café da manhã', 'Cafezinho', 'Cerveja', 'Chopp', 'Doces', 'Lanche', 'Outras bebidas alcoólicas', 'Refeição', 'Refrigerante e água mineral', 'Sorvete', 'Vinho', 'Alimentação no domicílio', 'Açúcares e derivados', 'Açúcar cristal', 'Açúcar demerara', 'Açúcar refinado', 'Balas', 'Chocolate e achocolatado em pó', 'Chocolate em barra e bombom', 'Doce de frutas em pasta', 'Doce de leite', 'Gelatina', 'Aves e ovos', 'Frango em pedaços', 'Frango inteiro', 'Ovo de galinha', 'Bebidas e infusões', 'Açaí (emulsão)', 'Café moído', 'Café solúvel', 'Chá', 'Chá mate (erva mate)', 'Polpa de açaí', 'Polpa de fruta (congelada)', 'Suco de frutas', 'Suco em pó', 'Carnes', 'Acém', 'Alcatra', 'Capa de filé', 'Carne de carneiro', 'Carne de porco', 'Chã de dentro', 'Contrafilé', 'Costela', 'Cupim', 'Fígado', 'Filé-mignon', 'Lagarto comum', 'Lagarto plano', 'Lagarto

In [5]:
# Executando a busca
resultados = search_keywords_in_rss(rss_feed_url, keywords)

Analisando feed: https://g1.globo.com/rss/g1/
Foram encontrados 100 itens no feed.
Analisando feed: https://feeds.folha.uol.com.br/emcimadahora/rss091.xml
Foram encontrados 100 itens no feed.
Analisando feed: https://www.bbc.com/portuguese/index.xml
Foram encontrados 15 itens no feed.
Analisando feed: https://exame.com/feed/
Foram encontrados 25 itens no feed.
Analisando feed: https://www.cartacapital.com.br/feed/
Foram encontrados 20 itens no feed.
Analisando feed: https://www.istoedinheiro.com.br/feed/
Foram encontrados 100 itens no feed.
Analisando feed: https://www.infomoney.com.br/feed/
Foram encontrados 10 itens no feed.
Analisando feed: https://www.jovempan.com.br/feed
Foram encontrados 80 itens no feed.
Analisando feed: https://economia.ig.com.br/rss.xml
Foram encontrados 51 itens no feed.


In [12]:
# Exibindo resultados
if resultados:
    print(f"Encontrados {len(resultados)} resultados:")
    for res in resultados:
        print(f"\nTítulo: {res['title']}\nLink: {res['link']}\nFonte: {res['feed_url']}\nData de publicação: {res['pub_date']}\nPalavra-chave: {res['matched_keyword']}")
else:
    print("Nenhuma notícia encontrada com as palavras-chave fornecidas.")


Encontrados 21 resultados:

Título: Reajuste anual no valor de medicamentos já está valendo; veja alternativas para economizar
Link: https://g1.globo.com/sp/presidente-prudente-regiao/noticia/2025/03/31/reajuste-anual-no-valor-de-medicamentos-ja-esta-valendo-veja-alternativas-para-economizar.ghtml
Fonte: https://g1.globo.com/rss/g1/
Data de publicação: Mon, 31 Mar 2025 19:50:38 -0000
Palavra-chave: inflação

Título: Justiça nega prisão domiciliar a pediatra acusado de estupros de crianças pela segunda vez 
Link: https://g1.globo.com/pb/paraiba/noticia/2025/03/31/justica-nega-prisao-domiciliar-a-pediatra-acusado-de-estupros-de-criancas-pela-segunda-vez.ghtml
Fonte: https://g1.globo.com/rss/g1/
Data de publicação: Mon, 31 Mar 2025 19:41:46 -0000
Palavra-chave: Cerveja

Título: Homem sofre queimaduras em 40% do corpo após companheiro jogar perfume e atear fogo na vítima
Link: https://g1.globo.com/sp/bauru-marilia/noticia/2025/03/31/homem-sofre-queimaduras-em-40percent-do-corpo-apos-compan

In [41]:
## Nova função para atualizar o CSV com as notícias encontradas - Aprovada
# Atualiza o arquivo CSV com as notícias encontradas, evitando duplicatas
import os
import pandas as pd

def update_noticias_csv(resultados):
    """
    Atualiza o arquivo CSV com novas notícias a partir da lista de dicionários 'resultados',
    incorporando colunas para classificação.

    A verificação de duplicidade é feita usando uma chave composta: (feed_url, pub_date, title).

    Parâmetros:
      resultados (list): Lista de dicionários, onde cada dicionário representa uma notícia.

    Retorna:
      df (DataFrame): DataFrame atualizado com as notícias e classificações.
    """
    
    # Define o caminho relativo do arquivo CSV
    csv_filepath = os.path.join('data', 'noticias', 'noticias.csv')
    
    # Garante que a pasta para o arquivo CSV exista
    os.makedirs(os.path.dirname(csv_filepath), exist_ok=True)
    
    # Colunas de classificação que serão incorporadas
    classification_columns = [
        "1. O artigo aborda o tema da inflação?",
        "2. O artigo apresenta uma perspectiva positiva para a economia, indicando uma queda na inflação geral?",
        "3. O artigo aborda especificamente a inflação de alimentos?",
        "4. O artigo apresenta uma perspectiva positiva para a inflação dos alimentos, indicando uma queda nesse setor?"
    ]
    
    # Se o arquivo já existe, carregamos os dados existentes; caso contrário, cria-se um DataFrame vazio com as colunas necessárias
    if os.path.exists(csv_filepath):
        df = pd.read_csv(csv_filepath)
    else:
        df = pd.DataFrame(columns=[
            'title', 'description', 'link', 'pub_date', 'feed_url', 'matched_keyword'
        ] + classification_columns)
    
    # Garante que todas as colunas de classificação existam no DataFrame; se não, adiciona-as com valores vazios
    for col in classification_columns:
        if col not in df.columns:
            df[col] = ""
    
    # Cria um conjunto de chaves compostas (feed_url, pub_date, title) para as notícias já cadastradas
    existing_keys = set(df.apply(lambda row: (row['feed_url'], row['pub_date'], row['title']), axis=1))
    
    new_entries = []  # Lista para armazenar as novas notícias a serem adicionadas
    
    for noticia in resultados:
        # Define a chave composta para a notícia atual
        composite_key = (noticia['feed_url'], noticia['pub_date'], noticia['title'])
        
        # Se a notícia ainda não está cadastrada, adiciona-a na lista
        if composite_key not in existing_keys:
            # Insere as colunas de classificação com valores vazios (ainda não classificadas)
            for col in classification_columns:
                noticia[col] = ""
            new_entries.append(noticia)
            existing_keys.add(composite_key)
        else:
            print(f"Notícia já cadastrada: {noticia['title']}")
    
    if new_entries:
        # Converte as novas entradas para um DataFrame e concatena com os dados existentes
        df_new = pd.DataFrame(new_entries)
        df = pd.concat([df, df_new], ignore_index=True)
        
        # Salva o DataFrame atualizado de volta no arquivo CSV
        df.to_csv(csv_filepath, index=False)
        print(f"{len(new_entries)} novas notícias adicionadas.")
    else:
        print("Nenhuma nova notícia foi encontrada para adicionar.")
    
    return df


In [42]:
# Exemplo de uso para atualização do arquivo de notícias
# Supondo que 'resultados' seja a lista de notícias obtidas da função de busca
resultados = search_keywords_in_rss(rss_feed_url, keywords)
df_atualizado = update_noticias_csv(resultados)

Analisando feed: https://g1.globo.com/rss/g1/
Foram encontrados 100 itens no feed.
Analisando feed: https://feeds.folha.uol.com.br/emcimadahora/rss091.xml
Foram encontrados 100 itens no feed.
Analisando feed: https://www.bbc.com/portuguese/index.xml
Foram encontrados 15 itens no feed.
Analisando feed: https://exame.com/feed/
Foram encontrados 25 itens no feed.
Analisando feed: https://www.cartacapital.com.br/feed/
Foram encontrados 20 itens no feed.
Analisando feed: https://www.istoedinheiro.com.br/feed/
Foram encontrados 100 itens no feed.
Analisando feed: https://www.infomoney.com.br/feed/
Foram encontrados 10 itens no feed.
Analisando feed: https://www.jovempan.com.br/feed
Foram encontrados 80 itens no feed.
Analisando feed: https://economia.ig.com.br/rss.xml
Foram encontrados 51 itens no feed.
Notícia já cadastrada: Cinco dicas para você proteger o meio ambiente e manter a praia limpa
Notícia já cadastrada: Policial da Core atacado por bandidos foi morto com cinco tiros de fuzil 
N

classificador-ia

In [43]:
# carregar o CSV atualizado
df = pd.read_csv(csv_filepath) 
df

,title,description,link,pub_date,feed_url,matched_keyword,1. O artigo aborda o tema da inflação?,"2. O artigo apresenta uma perspectiva positiva para a economia, indicando uma queda na inflação geral?",3. O artigo aborda especificamente a inflação de alimentos?,"4. O artigo apresenta uma perspectiva positiva para a inflação dos alimentos, indicando uma queda nesse setor?"
0,Reajuste anual no valor de medicamentos já est...,"<img src=""https://s2-g1.glbimg.com/pgn9BIW1...",https://g1.globo.com/sp/presidente-prudente-re...,"Mon, 31 Mar 2025 19:50:38 -0000",https://g1.globo.com/rss/g1/,inflação,NaN,NaN,NaN,NaN
1,Justiça nega prisão domiciliar a pediatra acus...,"<img src=""https://s2-g1.glbimg.com/ga75GDhj...",https://g1.globo.com/pb/paraiba/noticia/2025/0...,"Mon, 31 Mar 2025 19:41:46 -0000",https://g1.globo.com/rss/g1/,Cerveja,NaN,NaN,NaN,NaN
2,Homem sofre queimaduras em 40% do corpo após c...,"<img src=""https://s2-g1.glbimg.com/EftfoMrL...",https://g1.globo.com/sp/bauru-marilia/noticia/...,"Mon, 31 Mar 2025 19:18:57 -0000",https://g1.globo.com/rss/g1/,bebidas,NaN,NaN,NaN,NaN
3,Idoso é preso suspeito de estuprar criança em ...,"<img src=""https://s2-g1.glbimg.com/zwo37X-q...",https://g1.globo.com/ro/rondonia/noticia/2025/...,"Mon, 31 Mar 2025 19:05:13 -0000",https://g1.globo.com/rss/g1/,Doces,NaN,NaN,NaN,NaN
4,Santarém busca fortalecimento de políticas púb...,"<img src=""https://s2-g1.glbimg.com/mfo5791W...",https://g1.globo.com/pa/santarem-regiao/notici...,"Mon, 31 Mar 2025 18:52:31 -0000",https://g1.globo.com/rss/g1/,alimentação,NaN,NaN,NaN,NaN
5,Corpo de estudante de 19 anos que se afogou en...,"<img src=""https://s2-g1.glbimg.com/0HxgQwmZ...",https://g1.globo.com/pi/piaui/noticia/2025/03/...,"Mon, 31 Mar 2025 18:43:06 -0000",https://g1.globo.com/rss/g1/,Sal,NaN,NaN,NaN,NaN
6,Festa da Goiaba começa na sexta-feira (4) no D...,"<img src=""https://s2-g1.glbimg.com/SAptqubF...",https://g1.globo.com/df/distrito-federal/o-que...,"Mon, 31 Mar 2025 18:34:12 -0000",https://g1.globo.com/rss/g1/,Doces,NaN,NaN,NaN,NaN
7,Público pode trocar ingressos para o Suzano Mu...,"<img src=""https://s2-g1.glbimg.com/_zfuqrns...",https://g1.globo.com/sp/mogi-das-cruzes-suzano...,"Mon, 31 Mar 2025 18:12:36 -0000",https://g1.globo.com/rss/g1/,Leite em pó,NaN,NaN,NaN,NaN
8,Mulher é presa por suspeita de matar companhei...,"<img src=""https://s2-g1.glbimg.com/jLjqdlmv...",https://g1.globo.com/ap/amapa/noticia/2025/03/...,"Mon, 31 Mar 2025 17:11:46 -0000",https://g1.globo.com/rss/g1/,Peito,NaN,NaN,NaN,NaN
9,"Após disparada, pobreza na Argentina cai de 41...",\nApós um ano de sucessivas reduções no índice...,https://redir.folha.com.br/redir/online/emcima...,31 Mar 2025 16:30:00 -0300,https://feeds.folha.uol.com.br/emcimadahora/rs...,inflação,NaN,NaN,NaN,NaN


In [ ]:
from dotenv import load_dotenv
import os
import openai

# Carrega as variáveis definidas em .env
load_dotenv()

# Obtém a chave da API da OpenAI
api_key = os.getenv("OPENAI_API_KEY")

# Verifica se a chave da API foi carregada corretamente
if api_key is None:
    raise ValueError("A chave da API da OpenAI não foi encontrada. Verifique seu arquivo .env.")

# Inicializa o cliente da OpenAI
client = openai.OpenAI(api_key=api_key)



import os
import time
import pandas as pd
## Nova função para classificar os artigos
# Função de classificação, conforme sua implementação (certifique-se de já tê-la definida)
def classificar_artigo(artigo):
    """
    Função que recebe um dicionário com os dados do artigo e chama a API da OpenAI para classificar.
    Retorna o dicionário atualizado com as classificações.
    """
    system_message = (
        "Como economista especializado em inflação de alimentos, sua tarefa é analisar artigos de jornais e classificar "
        "conforme as seguintes perguntas:\n"
        "1. O artigo aborda o tema da inflação? (Responda com 'Sim' ou 'Não')\n"
           "- Se 'Sim':\n"
           "2. O artigo apresenta uma perspectiva positiva para a economia, indicando uma queda na inflação geral? "
           "(Responda com 'Sim', 'Não' ou 'Não se aplica')\n"
        "3. O artigo aborda especificamente a inflação de alimentos? (Responda com 'Sim', 'Não' ou 'Não se aplica')\n"
           "- Se 'Sim':\n"
           "4. O artigo apresenta uma perspectiva positiva para a inflação dos alimentos, indicando uma queda nesse setor? "
           "(Responda com 'Sim', 'Não' ou 'Não se aplica')\n"
        "Por favor, utilize apenas as palavras 'Sim', 'Não' ou 'Não se aplica' para responder a cada pergunta. "
        "Não é necessário justificar as respostas. \n"
        "Em caso de informações insuficientes no artigo para responder a alguma pergunta, responda 'Não'. \n"
        "Utilize apenas texto puro, sem aplicar estilos de formatação.\n"
        "Retorne a resposta com cada linha correspondendo a uma pergunta"
    )
    
    # Exemplo: usando apenas o título para a classificação (você pode incluir mais dados se necessário)
    user_message = f"Classifique:\nTítulo: {artigo['title']}\n"
    
    try:
        resposta = client.chat.completions.create(
            model="gpt-4o-mini",  # ou o modelo que você estiver utilizando
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ],
            temperature=0
        )
    except Exception as e:
        print("Erro ao chamar a API:", e)
        return {}
    
    conteudo = resposta.choices[0].message.content
    
    # Define as chaves das perguntas que queremos extrair
    perguntas = {
        "1. O artigo aborda o tema da inflação?": None,
        "2. O artigo apresenta uma perspectiva positiva para a economia, indicando uma queda na inflação geral?": None,
        "3. O artigo aborda especificamente a inflação de alimentos?": None,
        "4. O artigo apresenta uma perspectiva positiva para a inflação dos alimentos, indicando uma queda nesse setor?": None
    }
    
    # Processa a resposta da API (assumindo que ela retorne uma resposta com cada linha correspondendo a uma pergunta)
    for i, linha in enumerate(conteudo.strip().splitlines(), start=1):
        resposta_linha = linha.split('. ', 1)[-1].strip()
        chave_pergunta = list(perguntas.keys())[i - 1]
        perguntas[chave_pergunta] = resposta_linha
    
    # Atualiza o dicionário do artigo com as respostas
    artigo.update(perguntas)
    return artigo

import os
import time
import pandas as pd

def processar_classificacao_csv():
    """
    Carrega o arquivo 'data/noticias/noticias.csv', filtra os artigos que ainda não foram classificados 
    (verificando se a coluna "1. O artigo aborda o tema da inflação?" está vazia), classifica cada artigo 
    usando a função classificar_artigo e persiste as atualizações no CSV.
    """
    # Define o caminho relativo para o CSV
    csv_filepath = os.path.join('data', 'noticias', 'noticias.csv')
    
    # Verifica se o arquivo existe
    if not os.path.exists(csv_filepath):
        print("Arquivo CSV não encontrado:", csv_filepath)
        return
    
    # Carrega o DataFrame forçando a leitura das colunas como string (dtype=str)
    df = pd.read_csv(csv_filepath, dtype=str)
    
    # Lista de colunas de classificação desejadas
    col_classificacao = [
        "1. O artigo aborda o tema da inflação?",
        "2. O artigo apresenta uma perspectiva positiva para a economia, indicando uma queda na inflação geral?",
        "3. O artigo aborda especificamente a inflação de alimentos?",
        "4. O artigo apresenta uma perspectiva positiva para a inflação dos alimentos, indicando uma queda nesse setor?"
    ]
    
    # Garante que todas as colunas de classificação existam no DataFrame e preenche valores ausentes com ""
    for col in col_classificacao:
        if col not in df.columns:
            df[col] = ""
        else:
            df[col] = df[col].fillna("").astype(str)
    
    # Itera sobre os artigos que ainda não foram classificados
    for index, row in df.iterrows():
        # Se a coluna "1. O artigo aborda o tema da inflação?" estiver vazia, significa que o artigo ainda não foi classificado
        if row[col_classificacao[0]] == "":
            artigo = row.to_dict()
            print(f"Classificando artigo: {artigo['title']}")
            
            # Chama a função de classificação (certifique-se de que a função classificar_artigo já esteja definida e funcionando)
            artigo_classificado = classificar_artigo(artigo)
            
            # Atualiza as colunas de classificação no DataFrame com os valores retornados (convertendo para string se necessário)
            for col in col_classificacao:
                valor = artigo_classificado.get(col, "")
                df.at[index, col] = str(valor)

            # Imprime o resultado da classificação para o artigo atual
            print(f"Artigo '{artigo['title']}' classificado como:")
            for col in col_classificacao:
                print(f"  {col}: {df.at[index, col]}")
            
            # Pausa para evitar sobrecarga na API
            time.sleep(1)
    
    # Salva as atualizações no mesmo arquivo CSV
    df.to_csv(csv_filepath, index=False, encoding='utf-8')
    print("Classificação atualizada no arquivo CSV.")

# Exemplo de uso:
processar_classificacao_csv()



Classificando artigo: Reajuste anual no valor de medicamentos já está valendo; veja alternativas para economizar
Artigo 'Reajuste anual no valor de medicamentos já está valendo; veja alternativas para economizar' classificado como:
  1. O artigo aborda o tema da inflação?: Não
  2. O artigo apresenta uma perspectiva positiva para a economia, indicando uma queda na inflação geral?: Não se aplica
  3. O artigo aborda especificamente a inflação de alimentos?: Não
  4. O artigo apresenta uma perspectiva positiva para a inflação dos alimentos, indicando uma queda nesse setor?: Não se aplica
Classificando artigo: Justiça nega prisão domiciliar a pediatra acusado de estupros de crianças pela segunda vez 
Artigo 'Justiça nega prisão domiciliar a pediatra acusado de estupros de crianças pela segunda vez ' classificado como:
  1. O artigo aborda o tema da inflação?: Não
  2. O artigo apresenta uma perspectiva positiva para a economia, indicando uma queda na inflação geral?: Não se aplica
  3. O 

Classificador - abordagem antiga

In [8]:
# Exibindo resultados
if resultados:
    print(f"Encontrados {len(resultados)} resultados:")
    for res in resultados:
        print(f"\nTítulo: {res['title']}\nDescrição: {res['description']}\nLink: {res['link']}\nFonte: {res['feed_url']}\nData de publicação: {res['pub_date']}\nPalavra-chave: {res['matched_keyword']}")
else:
    print("Nenhuma notícia encontrada com as palavras-chave fornecidas.")




Encontrados 38 resultados:

Título: Mulher é presa suspeita de matar homem com facada no peito após ele se negar a pagar programa sexual, diz polícia
Descrição:    <img src="https://s2-g1.glbimg.com/T1NXBNfhB4IW6_Z0CtSmbiiFvls=/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2025/O/u/soTSJsQGAf21JgbXPgHQ/whatsapp-image-2025-03-27-at-12.58.20.jpeg" /><br />     Crime aconteceu em 2022, em Gurupi. Suspeita e homem teriam se desentendido e ela o atingiu com um golpe no peito. Mulher é presa em Gurupi por suspeita de homicídio em 2022
Uma mulher foi presa nesta quinta-feira (27) suspeita de esfaquear e matar um homem em Gurupi, no sul do estado. A vítima, Eliakim Ferreira Vieira, teria se negado a pagar um programa sexual com a suspeita, o que teria motivado o homicídio.
📱 Participe do canal do g1 TO no WhatsApp e receba as notícias no celular.
O crime aconteceu no dia 17 de maio de 2022 e, de acordo com a Polícia Civil, eles teriam se desentendido por causa do 

In [9]:
# Fazer um slice e pegar os 5 primeiros resultados e guardar em resultados_1
resultados_1 = resultados[:5]
print(resultados_1)

[{'title': 'Mulher é presa suspeita de matar homem com facada no peito após ele se negar a pagar programa sexual, diz polícia', 'description': '   <img src="https://s2-g1.glbimg.com/T1NXBNfhB4IW6_Z0CtSmbiiFvls=/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2025/O/u/soTSJsQGAf21JgbXPgHQ/whatsapp-image-2025-03-27-at-12.58.20.jpeg" /><br />     Crime aconteceu em 2022, em Gurupi. Suspeita e homem teriam se desentendido e ela o atingiu com um golpe no peito. Mulher é presa em Gurupi por suspeita de homicídio em 2022\nUma mulher foi presa nesta quinta-feira (27) suspeita de esfaquear e matar um homem em Gurupi, no sul do estado. A vítima, Eliakim Ferreira Vieira, teria se negado a pagar um programa sexual com a suspeita, o que teria motivado o homicídio.\n📱 Participe do canal do g1 TO no WhatsApp e receba as notícias no celular.\nO crime aconteceu no dia 17 de maio de 2022 e, de acordo com a Polícia Civil, eles teriam se desentendido por causa do pagamento. A m

In [11]:
from dotenv import load_dotenv
import os
import openai

# Carrega as variáveis definidas em .env
load_dotenv()

# Obtém a chave da API da OpenAI
api_key = os.getenv("OPENAI_API_KEY")

# Verifica se a chave da API foi carregada corretamente
if api_key is None:
    raise ValueError("A chave da API da OpenAI não foi encontrada. Verifique seu arquivo .env.")

# Inicializa o cliente da OpenAI
client = openai.OpenAI(api_key=api_key)

#print("Cliente da OpenAI inicializado com sucesso!")
#print(client)


In [15]:
def classificar_artigo(artigo):
    """
    Função que recebe um dicionário com os dados do artigo e chama a API da OpenAI para classificar.
    
    Parâmetros do artigo (dicionário):
      - title: título do artigo
      - description: descrição do artigo
      - link: link do artigo
      - feed_url: fonte do artigo
      - pub_date: data de publicação
      - matched_keyword: palavra-chave que originou o artigo
      
    Retorna:
      - O dicionário do artigo atualizado com as classificações extraídas da resposta da API.
    """
    # Mensagem do sistema com as instruções de classificação
    system_message = (
        "Como economista especializado em inflação de alimentos, sua tarefa é analisar artigos de jornais e classificá-los conforme as seguintes perguntas:\n"
        "1. O artigo aborda o tema da inflação? (Responda com 'Sim' ou 'Não')\n"
           "- Se 'Sim':\n"
           "2. O artigo apresenta uma perspectiva positiva para a economia, indicando uma queda na inflação geral? (Responda com 'Sim', 'Não' ou 'Não se aplica')\n"
           "3. O artigo aborda especificamente a inflação de alimentos? (Responda com 'Sim', 'Não' ou 'Não se aplica')\n"
              "- Se 'Sim':\n"
              "4. O artigo apresenta uma perspectiva positiva para a inflação dos alimentos, indicando uma queda nesse setor? (Responda com 'Sim', 'Não' ou 'Não se aplica')\n"
        "Por favor, utilize apenas as palavras 'Sim', 'Não' ou 'Não se aplica' para responder a cada pergunta. Não é necessário justificar as respostas. \n"
        "Em caso de informações insuficientes no artigo para responder a alguma pergunta, responda 'Não'. \n"
        "Utilize apenas texto puro, sem aplicar estilos de formatação."
    )
    
    # Mensagem do usuário com os dados do artigo
    # Mensagem do usuário com os dados do artigo
    # user_message = (
    #     f"Classifique:\n"
    #     f"Título: {artigo['title']}\n"
    #     f"Descrição: {artigo['description']}\n"
    #     f"Link: {artigo['link']}\n"
    #     f"Fonte: {artigo['feed_url']}\n"
    #     f"Data de publicação: {artigo['pub_date']}\n"
    #     f"Palavra-chave: {artigo['matched_keyword']}"
    # )
    user_message = (
        f"Classifique:\n"
        f"Título: {artigo['title']}\n"
    )
    
    try:
        resposta = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ],
            temperature=0  # para respostas mais determinísticas
        )
        #print("Resposta da API:", resposta)
    except Exception as e:
        print("Erro ao chamar a API:", e)
        return {}
    
    # Extrai o conteúdo da resposta
    conteudo = resposta.choices[0].message.content
    #print("Conteúdo da resposta:", conteudo)
    
    # Mapeamento das perguntas para suas respectivas chaves
    perguntas = {
        "1. O artigo aborda o tema da inflação?": None,
        "2. O artigo apresenta uma perspectiva positiva para a economia, indicando uma queda na inflação geral?": None,
        "3. O artigo aborda especificamente a inflação de alimentos?": None,
        "4. O artigo apresenta uma perspectiva positiva para a inflação dos alimentos, indicando uma queda nesse setor?": None
    }
    
    # Divide o conteúdo da resposta em linhas e preenche o dicionário de perguntas com as respostas correspondentes
    for i, linha in enumerate(conteudo.strip().splitlines(), start=1):
        resposta_linha = linha.split('. ', 1)[-1].strip()
        chave_pergunta = list(perguntas.keys())[i - 1]
        perguntas[chave_pergunta] = resposta_linha
    
    # Apensa os resultados (as classificações) ao dicionário do artigo
    artigo.update(perguntas)
    
    # Retorna apenas o dicionário do artigo, já atualizado com as classificações
    return artigo


In [16]:
artigo = resultados[30]
#imprimir o tipo da variável artigo
# print(type(artigo))
# print(artigo)

classificacao = classificar_artigo(artigo)
print(classificacao)

{'title': '8 riscos e benefícios de consumir sal antes do treino', 'description': '<p>Indrediente é um eletrólito essencial para o funcionamento do organismo, mas deve ser usado com cautela</p>\n', 'link': 'https://jovempan.com.br/edicase/8-riscos-e-beneficios-de-consumir-sal-antes-do-treino.html', 'pub_date': 'Thu, 27 Mar 2025 21:30:00 +0000', 'feed_url': 'https://www.jovempan.com.br/feed', 'matched_keyword': 'Sal', '1. O artigo aborda o tema da inflação?': 'Não', '2. O artigo apresenta uma perspectiva positiva para a economia, indicando uma queda na inflação geral?': None, '3. O artigo aborda especificamente a inflação de alimentos?': None, '4. O artigo apresenta uma perspectiva positiva para a inflação dos alimentos, indicando uma queda nesse setor?': None}


In [182]:
feed_url = artigo.get('feed_url')
print(feed_url)

https://www.jovempan.com.br/feed


In [159]:
def salvar_resultados_csv(artigos, classificacoes, nome_arquivo="resultados_classificacao.csv"):
    """
    Recebe a lista de artigos e as classificações correspondentes, e salva em um arquivo CSV.
    As colunas do CSV serão:
      - Título
      - Descrição
      - Link
      - Fonte
      - Data de publicação
      - Palavra-chave
      - O artigo trata de inflação?
      - É um artigo positiva para a economia (inflação em queda)?
      - O artigo trata de inflação de alimentos?
      - É um artigo positiva para a inflação dos alimentos (inflação em queda)?
    """
    linhas = []
    for artigo, classif in zip(artigos, classificacoes):
        linha = {
            "Título": artigo["title"],
            "Descrição": artigo["description"],
            "Link": artigo["link"],
            "Fonte": artigo["feed_url"],
            "Data de publicação": artigo["pub_date"],
            "Palavra-chave": artigo["matched_keyword"],
            "O artigo trata de inflação?": classif.get("O artigo trata de inflação?", ""),
            "É um artigo positiva para a economia (inflação em queda)?": classif.get("É um artigo positiva para a economia (inflação em queda)?", ""),
            "O artigo trata de inflação de alimentos?": classif.get("O artigo trata de inflação de alimentos?", ""),
            "É um artigo positiva para a inflação dos alimentos (inflação em queda)?": classif.get("É um artigo positiva para a inflação dos alimentos (inflação em queda)?", "")
        }
        linhas.append(linha)
        
    df = pd.DataFrame(linhas)
    df.to_csv(nome_arquivo, index=False, encoding='utf-8')
    print(f"Resultados salvos no arquivo {nome_arquivo}")

def processar_artigos(artigos):
    """
    Itera sobre uma lista de artigos, chama a função de classificação para cada um e salva o resultado num CSV.
    
    Parâmetro:
      - artigos: lista de dicionários, onde cada dicionário contém as chaves:
          title, description, link, feed_url, pub_date, matched_keyword
    """
    classificacoes = []
    for idx, artigo in enumerate(artigos, start=1):
        print(f"Processando artigo {idx}/{len(artigos)}: {artigo['title']}")
        classif = classificar_artigo(artigo)
        classificacoes.append(classif)
        # Pausa para evitar sobrecarga na API
        time.sleep(1)
    

    salvar_resultados_csv(artigos, classificacoes)

    
artigos_classificados = processar_artigos(resultados)


Processando artigo 1/36: Homem morre e outro fica ferido após tentarem fugir e entrarem e confronto com policiais em Gurupi, diz PM
Resposta da API: ChatCompletion(id='chatcmpl-BFoLcMSu5QuUEhE0QVoY4F5lva8l3', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. Não\n2. Não se aplica\n3. Não se aplica\n4. Não se aplica', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1743108156, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_b376dfbbd5', usage=CompletionUsage(completion_tokens=22, prompt_tokens=1120, total_tokens=1142, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=1024)))
Conteúdo da resposta: 1. Não
2. Não se aplica
3. Não se aplica
4. Não se aplica

In [163]:
print(artigos_classificados)

None


In [2]:
import pandas as pd
# abrir resultados_classificacao.csv e exibir os resultados
df = pd.read_csv('resultados_classificacao.csv')
df

,Título,Descrição,Link,Fonte,Data de publicação,Palavra-chave,O artigo trata de inflação?,É um artigo positiva para a economia (inflação em queda)?,O artigo trata de inflação de alimentos?,É um artigo positiva para a inflação dos alimentos (inflação em queda)?
0,Homem morre e outro fica ferido após tentarem ...,"<img src=""https://s2-g1.glbimg.com/7wohoWSZ...",https://g1.globo.com/to/tocantins/noticia/2025...,https://g1.globo.com/rss/g1/,"Thu, 27 Mar 2025 20:04:18 -0000",bebidas,NaN,NaN,NaN,NaN
1,Bertioga implementa policiamento ciclístico; e...,"<img src=""https://s2-g1.glbimg.com/BK57TjL_...",https://g1.globo.com/sp/santos-regiao/noticia/...,https://g1.globo.com/rss/g1/,"Thu, 27 Mar 2025 20:00:14 -0000",Pescados,NaN,NaN,NaN,NaN
2,Justiça determina que prefeitura de Rio Branco...,"<img src=""https://s2-g1.glbimg.com/TTxMR9Sz...",https://g1.globo.com/ac/acre/noticia/2025/03/2...,https://g1.globo.com/rss/g1/,"Thu, 27 Mar 2025 19:59:19 -0000",alimentação,NaN,NaN,NaN,NaN
3,"Antes de atirar no rosto da tia, sobrinho teve...","<img src=""https://s2-g1.glbimg.com/2sMAriCs...",https://g1.globo.com/ac/acre/noticia/2025/03/2...,https://g1.globo.com/rss/g1/,"Thu, 27 Mar 2025 19:44:33 -0000",bebidas,NaN,NaN,NaN,NaN
4,Irmão do suspeito de atirar contra pessoas em ...,"<img src=""https://s2-g1.glbimg.com/SiZAmEwf...",https://g1.globo.com/sp/sorocaba-jundiai/notic...,https://g1.globo.com/rss/g1/,"Thu, 27 Mar 2025 18:59:52 -0000",Pá,NaN,NaN,NaN,NaN
5,'Tradwife dos LGBTs': Influencer conquista púb...,\nSanduíche de berinjela empanada no pão ciaba...,https://redir.folha.com.br/redir/online/emcima...,https://feeds.folha.uol.com.br/emcimadahora/rs...,27 Mar 2025 15:30:00 -0300,Alcatra,NaN,NaN,NaN,NaN
6,"Remédios devem ficar até 5,06% mais caros a pa...","\nA partir da próxima segunda-feira (31), os p...",https://redir.folha.com.br/redir/online/emcima...,https://feeds.folha.uol.com.br/emcimadahora/rs...,27 Mar 2025 15:23:00 -0300,inflação,NaN,NaN,NaN,NaN
7,"Ovo, café e tomate puxam inflação dos alimento...","\nA inflação dos alimentos no <a href=""https:/...",https://redir.folha.com.br/redir/online/emcima...,https://feeds.folha.uol.com.br/emcimadahora/rs...,27 Mar 2025 13:35:00 -0300,inflação,NaN,NaN,NaN,NaN
8,"Despesa com Bolsa Família cai, estímulo do gas...","\nA despesa com o <a href=""https://www1.folha....",https://redir.folha.com.br/redir/online/emcima...,https://feeds.folha.uol.com.br/emcimadahora/rs...,27 Mar 2025 13:20:00 -0300,inflação,NaN,NaN,NaN,NaN
9,Cerveja sem álcool é eleita a 3ª melhor do paí...,\nEm meados de março terminou a 13ª edição do ...,https://redir.folha.com.br/redir/online/emcima...,https://feeds.folha.uol.com.br/emcimadahora/rs...,27 Mar 2025 11:58:00 -0300,Cerveja,NaN,NaN,NaN,NaN


# Nuvem de palavras

In [2]:
import nltk
from nltk.corpus import stopwords
import os

# Baixa o corpus de stopwords (se ainda não estiver baixado)
nltk.download('stopwords')

# Obtém a lista de stopwords em português
stopwords_pt = stopwords.words('portuguese')

# Define o caminho onde salvar o arquivo
caminho = r"C:\Users\marce\OneDrive - Ministério da Agricultura e Pecuária\LAB_DATA_MDA\20250325_alerta_inflacao\data\keywords\stopwords.txt"

# Certifica-se de que a pasta existe
os.makedirs(os.path.dirname(caminho), exist_ok=True)

# Salva as stopwords no arquivo, uma por linha
with open(caminho, 'w', encoding='utf-8') as f:
    for palavra in stopwords_pt:
        f.write(palavra + "\n")

print("Stopwords em português salvas com sucesso em:", caminho)



Stopwords em português salvas com sucesso em: C:\Users\marce\OneDrive - Ministério da Agricultura e Pecuária\LAB_DATA_MDA\20250325_alerta_inflacao\data\keywords\stopwords.txt


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marce\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# Importa as bibliotecas necessárias
import pandas as pd
import string
from collections import Counter
import nltk
from nltk.corpus import stopwords

# Se ainda não baixou as stopwords do NLTK, descomente a linha abaixo
nltk.download('stopwords')

# Define as stopwords em português
stopwords_pt = set(stopwords.words('portuguese'))

# Caminho do arquivo CSV
csv_path = "data/noticias/noticias.csv"

# Lê o CSV; força todas as colunas como string para evitar problemas de formatação
df = pd.read_csv(csv_path, dtype=str)

# Verifica se a coluna de interesse existe
if "1. O artigo aborda o tema da inflação?" not in df.columns:
    raise ValueError("Coluna '1. O artigo aborda o tema da inflação?' não encontrada no CSV.")

# Filtra apenas os artigos em que o tema é "Sim"
df_filtrado = df[df["1. O artigo aborda o tema da inflação?"] == "Sim"]

# Extrai os textos dos títulos
textos = df_filtrado["title"].tolist()

# Se existir a coluna 'description', adiciona também suas informações (removendo valores nulos)
if "description" in df_filtrado.columns:
    textos += df_filtrado["description"].dropna().tolist()

# Junta todos os textos em uma única string e converte para minúsculas
texto_completo = " ".join(textos).lower()

# Remove pontuação
texto_sem_pontuacao = texto_completo.translate(str.maketrans("", "", string.punctuation))

# Separa as palavras e remove as stopwords
palavras = texto_sem_pontuacao.split()
palavras_filtradas = [palavra for palavra in palavras if palavra not in stopwords_pt]

# Conta a frequência das palavras usando Counter
frequencias = Counter(palavras_filtradas)

# Converte o resultado para um DataFrame
df_frequencias = pd.DataFrame(frequencias.items(), columns=["Palavra", "Quantidade"])

# Ordena as palavras pela quantidade, de forma decrescente
df_frequencias = df_frequencias.sort_values(by="Quantidade", ascending=False).reset_index(drop=True)

# Exibe o DataFrame resultante
df_frequencias


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marce\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Palavra,Quantidade
0,governo,60
1,inflação,57
2,lula,45
3,diz,30
4,presidente,29
...,...,...
1943,banca,1
1944,alteração,1
1945,yoshio,1
1946,wagner,1
